In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *

In [4]:
x_train = []
y_train = []

label_map = {   "Black-grass"               :0,
                "Charlock"                  :1,
                "Cleavers"                  :2,
                "Common Chickweed"          :3,
                "Common wheat"              :4,
                "Fat Hen"                   :5,
                "Loose Silky-bent"          :6,
                "Maize"                     :7,
                "Scentless Mayweed"         :8,
                "Shepherds Purse"           :9,
                "Small-flowered Cranesbill" :10,
                "Sugar beet"                :11}

dim = 256

# Preparing training data
dirs = os.listdir("/input/plant_seedlings_classification/train/")
for k in tqdm(range(len(dirs))):    # Directory
    files = os.listdir("/input/plant_seedlings_classification/train/{}".format(dirs[k]))
    for f in range(len(files)):     # Files
        img = cv2.imread('/input/plant_seedlings_classification/train/{}/{}'.format(dirs[k], files[f]))
        targets = np.zeros(12)
        targets[label_map[dirs[k]]] = 1 
        x_train.append(cv2.resize(img, (dim, dim)))
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)

print(x_train.shape)
print(y_train.shape)

#x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.01, random_state=42)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.16, random_state=42) # Want a balanced split for all the classes
for train_index, test_index in sss.split(x_train, y_train):
    print("Using {} for training and {} for validation".format(len(train_index), len(test_index)))
    x_train, x_valid = x_train[train_index], x_train[test_index]
    y_train, y_valid = y_train[train_index], y_train[test_index]

100%|██████████| 12/12 [00:54<00:00,  4.83s/it]


(4750, 256, 256, 3)
(4750, 12)


In [ ]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range = 40,
        horizontal_flip=True)
                                  
val_datagen = ImageDataGenerator(rescale = 1./255)
                                      
weights = os.path.join('', '/output/weights.h5')

epochs = 100
learning_rate = 0.001
batch_size = 32

callbacks = [ EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
              ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]

base_model = Xception(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [51]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=val_datagen.flow(x_valid, y_valid, batch_size=batch_size), 
                    validation_steps=len(x_valid)/batch_size,
                    callbacks=callbacks,
                    epochs=20, 
                    verbose=1)

Epoch 1/20
125/124 [==============================] - 249s - loss: 0.0211 - acc: 0.9927 - val_loss: 0.1085 - val_acc: 0.9671
Epoch 2/20
125/124 [==============================] - 249s - loss: 0.0256 - acc: 0.9932 - val_loss: 0.1003 - val_acc: 0.9737
Epoch 3/20
125/124 [==============================] - 249s - loss: 0.0217 - acc: 0.9934 - val_loss: 0.0902 - val_acc: 0.9671
Epoch 4/20
125/124 [==============================] - 249s - loss: 0.0148 - acc: 0.9950 - val_loss: 0.1177 - val_acc: 0.9697
Epoch 5/20
125/124 [==============================] - 249s - loss: 0.0203 - acc: 0.9955 - val_loss: 0.1193 - val_acc: 0.9724
Epoch 6/20
125/124 [==============================] - 249s - loss: 0.0127 - acc: 0.9975 - val_loss: 0.1061 - val_acc: 0.9737
Epoch 7/20
125/124 [==============================] - 249s - loss: 0.0138 - acc: 0.9945 - val_loss: 0.0943 - val_acc: 0.9789
Epoch 8/20
125/124 [==============================] - 248s - loss: 0.0127 - acc: 0.9969 - val_loss: 0.1148 - val_acc: 0.9697


In [53]:
# ------ TESTING ------
x_test = []
df_test = pd.read_csv('/input/plant_seedlings_classification/sample_submission.csv')

for f, species in tqdm(df_test.values, miniters=100):
    img = cv2.imread('/input/plant_seedlings_classification/test/{}'.format(f))
    x_test.append(cv2.resize(img, (dim, dim)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)

x_test = x_test /255.0

if os.path.isfile(weights):
    model.load_weights(weights)

p_test = model.predict(x_test, verbose = 1)

preds = []
for i in range(len(p_test)):
    pos = np.argmax(p_test[i])
    preds.append(list(label_map.keys())[list(label_map.values()).index(pos)])


df_test['species'] = preds
df_test.to_csv('/output/submission.csv', index=False)

100%|██████████| 794/794 [00:16<00:00, 48.55it/s]


(794, 256, 256, 3)
